# Fiddler Simple Monitoring Quick Start Guide

Fiddler is the pioneer in enterprise Model Performance Management (MPM), offering a unified platform that enables Data Science, MLOps, Risk, Compliance, Analytics, and other LOB teams to **monitor, explain, analyze, and improve ML deployments at enterprise scale**. 
Obtain contextual insights at any stage of the ML lifecycle, improve predictions, increase transparency and fairness, and optimize business revenue.

---

You can start using Fiddler ***in minutes*** by following these 7 quick steps:

1. Imports
2. Connect to Fiddler
3. Upload a baseline dataset
4. Add metadata about your model with Fiddler
5. Set up Alerts and Notifications (Optional)
6. Publish production events
7. Get insights

**Don't have a Fiddler account? [Sign-up for a 14-day free trial](https://www.fiddler.ai/trial?utm_source=fiddler_docs&utm_medium=referral).**

## 1. Imports

In [1]:
!pip install -q fiddler-client==1.8.0

import numpy as np
import pandas as pd
import time as time
import fiddler as fdl

print(f"Running client version {fdl.__version__}")

Running client version 1.8.0


## 2. Connect to Fiddler

Before you can add information about your model with Fiddler, you'll need to connect using our API client.


---


**We need a few pieces of information to get started.**
1. The URL you're using to connect to Fiddler

In [2]:
URL = 'https://preprod.fiddler.ai' # Make sure to include the full URL (including https://). For example, https://abc.xyz.ai

2. Your organization ID
3. Your authorization token

Both of these can be found by clicking the URL you entered and navigating to the **Settings** page.

<table>
    <tr>
        <td><img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/1.png" /></td>
        <td><img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/2.png" /></td>
    </tr>
    <tr>
        <td><img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/3.png" /></td>
        <td><img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/4.png" /></td>
    </tr>
</table>

In [3]:
ORG_ID = 'preprod'
AUTH_TOKEN = '6lxdgyAZ3B2PNFxR3GZ7N4ao6As6UvicPQdamdaU13g'

Now just run the following code block to connect to the Fiddler API!

In [4]:
client = fdl.FiddlerApi(url=URL, org_id=ORG_ID, auth_token=AUTH_TOKEN)

Once you connect, you can create a new project by specifying a unique project ID in the client's `create_project` function.

In [6]:
PROJECT_ID = 'one_point_eight'

client.create_project(PROJECT_ID)

{'project_name': 'one_point_eight'}

You should now be able to see the newly created project on the UI.

<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/5.png" />
        </td>
    </tr>
</table>

## 3. Upload a baseline dataset

In this example, we'll be considering the case where we're a bank and we have **a model that predicts churn for our customers**.  
We want to know when our model's predictions start to drift—that is, **when churn starts to increase** within our customer base.
  
In order to get insights into the model's performance, **Fiddler needs a small  sample of data that can serve as a baseline** for making comparisons with data in production.


---


*For more information on how to design a baseline dataset, [click here](https://docs.fiddler.ai/docs/designing-a-baseline-dataset).*

In [7]:
PATH_TO_BASELINE_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/churn_baseline.csv'

baseline_df = pd.read_csv(PATH_TO_BASELINE_CSV)
baseline_df

,customer_id,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,predicted_churn,decision,churn
0,27acd1c2,545,Texas,Male,37,9,110483.86,1,1,1,127394.67,0.897202,high_risk,yes
1,27b36d0c,497,Texas,Female,55,7,131778.66,1,1,1,9972.64,0.997441,high_risk,yes
2,27b5360a,509,New York,Female,29,0,107712.57,2,1,1,92898.17,0.920563,high_risk,yes
3,27b5d650,743,Hawaii,Nonbinary,39,6,0.00,2,1,0,44265.28,0.779282,high_risk,yes
4,27b236a8,699,Florida,Female,25,8,0.00,2,1,1,52404.47,0.825474,high_risk,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,27b409ba,686,Texas,Male,39,3,129626.19,2,1,1,103220.56,0.760645,high_risk,yes
19996,27aaff96,446,Massachusetts,Female,45,10,125191.69,1,1,1,128260.86,0.216093,low_risk,no
19997,27ad3162,794,California,Male,35,6,0.00,2,1,1,68730.91,0.982021,high_risk,yes
19998,27b076ce,832,California,Male,61,2,0.00,1,0,1,127804.66,0.071598,low_risk,no


Fiddler uses this baseline dataset to keep track of important information about your data.
  
This includes **data types**, **data ranges**, and **unique values** for categorical variables.

---

You can construct a `DatasetInfo` object to be used as **a schema for keeping track of this information** by running the following code block.

In [8]:
dataset_info = fdl.DatasetInfo.from_dataframe(baseline_df, max_inferred_cardinality=100)
dataset_info

,column,dtype,count(possible_values),is_nullable,value_range
0,customer_id,STRING,,False,
1,creditscore,INTEGER,,False,350 - 850
2,geography,CATEGORY,6,False,
3,gender,CATEGORY,3,False,
4,age,INTEGER,,False,18 - 92
5,tenure,INTEGER,,False,0 - 10
6,balance,FLOAT,,False,"0.0 - 250,900.0"
7,numofproducts,INTEGER,,False,1 - 4
8,hascrcard,INTEGER,,False,0 - 1
9,isactivemember,INTEGER,,False,0 - 1


Then use the client's [upload_dataset](https://docs.fiddler.ai/reference/clientupload_dataset) function to send this information to Fiddler!
  
*Just include:*
1. A unique dataset ID
2. The baseline dataset as a pandas DataFrame
3. The [DatasetInfo](https://docs.fiddler.ai/reference/fdldatasetinfo) object you just created

In [9]:
DATASET_ID = 'churn_data'

client.upload_dataset(
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    dataset={
        'baseline': baseline_df
    },
    info=dataset_info
)

{'uuid': 'dcab31d2-640a-49bc-81a4-a24453d1e9b0',
 'name': 'Ingestion dataset Upload',
 'info': {'project_name': 'one_point_eight',
  'resource_name': 'churn_data',
  'resource_type': 'DATASET'},
 'status': 'SUCCESS',
 'progress': 100.0,
 'error_message': None,
 'error_reason': None}

If you click on your project, you should now be able to see the newly created dataset on the UI.

<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/6.png" />
        </td>
    </tr>
</table>

## 4. Add metadata about your model

Now it's time to add your model with Fiddler.  We do this by defining a [ModelInfo](https://docs.fiddler.ai/reference/fdlmodelinfo) object.


---


The [ModelInfo](https://docs.fiddler.ai/reference/fdlmodelinfo) object will contain some **information about how your model operates**.
  
*Just include:*
1. The **task** your model is performing (regression, binary classification, etc.)
2. The **target** (ground truth) column
3. The **output** (prediction) column
4. The **feature** columns
5. Any **metadata** columns
6. Any **decision** columns (these measures the direct business decisions made as result of the model's prediction)


In [11]:
# Specify task
model_task = 'binary'

if model_task == 'regression':
    model_task = fdl.ModelTask.REGRESSION
    
elif model_task == 'binary':
    model_task = fdl.ModelTask.BINARY_CLASSIFICATION

elif model_task == 'multiclass':
    model_task = fdl.ModelTask.MULTICLASS_CLASSIFICATION
    
elif model_task == 'ranking':
    model_task = fdl.ModelTask.RANKING

    
# Specify column types
features = ['geography', 'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard', 'isactivemember', 'estimatedsalary']
outputs = ['predicted_churn']
target = 'churn'
decision_cols = ['decision']
metadata_cols = ['customer_id']
    
# Generate ModelInfo
model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=dataset_info,
    dataset_id=DATASET_ID,
    model_task=model_task,
    features=features,
    outputs=outputs,
    target=target,
    categorical_target_class_details='yes',
    decision_cols=decision_cols, # Optional
    metadata_cols=metadata_cols, # Optional
    binary_classification_threshold=0.5 # Optional
)
model_info

Using binary_classification_threshold=0.5


,column,dtype,count(possible_values),is_nullable,value_range
0,churn,CATEGORY,2,False,
,column,dtype,count(possible_values),is_nullable,value_range
0,geography,CATEGORY,6,False,
1,gender,CATEGORY,3,False,
2,age,INTEGER,,False,18 - 92
3,tenure,INTEGER,,False,0 - 10
4,balance,FLOAT,,False,"0.0 - 250,900.0"
5,numofproducts,INTEGER,,False,1 - 4
6,hascrcard,INTEGER,,False,0 - 1
7,isactivemember,INTEGER,,False,0 - 1


Almost done! Now just specify a unique model ID and use the client's [add_model](https://docs.fiddler.ai/reference/clientadd_model) function to send this information to Fiddler.

In [12]:
MODEL_ID = 'churn_classifier'

client.add_model(
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    model_id=MODEL_ID,
    model_info=model_info,
)

On the project page, you should now be able to see the newly created model.

<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/7.png" />
        </td>
    </tr>
</table>

## 5. Set up Alerts and Notifications (Optional)

Fiddler Client API function [add_alert_rule](https://dash.readme.com/project/fiddler/v1.5/refs/clientadd_alert_rule) allow creating rules to receive email and pagerduty notifications when your data or model predictions deviates from it's expected behavior.

The rules can of **Data Drift, Performance, Data Integrity,** and **Service Metrics** types and they can be compared to **absolute** or **relative** values.

Please refer [our documentation](https://docs.fiddler.ai/docs/alerts) for more information on Alert Rules. 

---
  
Let's set up a few Alert Rules.

The following API call sets up a Data Integrity type rule which triggers an email notification when published events have 2 or more range violations in any 1 day bin for the ```numofproducts``` column.

In [13]:
notifications_config = client.build_notifications_config(
    emails = "name@google.com",
)

client.add_alert_rule(
    name = "Bank Churn Range Violation Alert1",
    project_id = PROJECT_ID,
    model_id = MODEL_ID,
    alert_type = fdl.AlertType.DATA_INTEGRITY,
    metric = fdl.Metric.RANGE_VIOLATION,
    bin_size = fdl.BinSize.ONE_DAY, 
    compare_to = fdl.CompareTo.RAW_VALUE,
    #compare_period = None,
    priority = fdl.Priority.HIGH,
    warning_threshold = 2,
    critical_threshold = 3,
    condition = fdl.AlertCondition.GREATER,
    column = "numofproducts",
    notifications_config = notifications_config
)

AlertRule(alert_rule_uuid='40bf4bd3-fc83-467e-af4a-0cc0336ffdbe', organization_name='preprod', project_id='one_point_eight', model_id='churn_classifier', name='Bank Churn Range Violation Alert1', alert_type=AlertType.DATA_INTEGRITY, metric=Metric.RANGE_VIOLATION, column='numofproducts', baseline_id='DEFAULT (churn_data)', priority=Priority.HIGH, compare_to=CompareTo.RAW_VALUE, compare_period=None, warning_threshold=2.0, critical_threshold=3.0, condition=AlertCondition.GREATER, bin_size=BinSize.ONE_DAY)

The following API call sets up a Performance type rule which triggers an email notification when precision metric is 5% higher than that from 1 hr bin one day ago.

In [14]:
notifications_config = client.build_notifications_config(
    emails = "name@google.com",
)
client.add_alert_rule(
    name = "Bank Churn Performance Alert",
    project_id = PROJECT_ID,
    model_id = MODEL_ID,
    alert_type = fdl.AlertType.PERFORMANCE,
    metric = fdl.Metric.PRECISION,
    bin_size = fdl.BinSize.ONE_HOUR, 
    compare_to = fdl.CompareTo.TIME_PERIOD,
    compare_period = fdl.ComparePeriod.ONE_DAY,
    warning_threshold = 0.05,
    critical_threshold = 0.1,
    condition = fdl.AlertCondition.GREATER,
    priority = fdl.Priority.HIGH,
    notifications_config = notifications_config
)

AlertRule(alert_rule_uuid='71fea5d2-5590-4276-921c-3d9bb8dfa191', organization_name='preprod', project_id='one_point_eight', model_id='churn_classifier', name='Bank Churn Performance Alert', alert_type=AlertType.PERFORMANCE, metric=Metric.PRECISION, column='', baseline_id='DEFAULT (churn_data)', priority=Priority.HIGH, compare_to=CompareTo.TIME_PERIOD, compare_period=ComparePeriod.ONE_DAY, warning_threshold=0.05, critical_threshold=0.1, condition=AlertCondition.GREATER, bin_size=BinSize.ONE_HOUR)

## 6. Publish production events

Information about your model is added to Fiddler and now it's time to start publishing some production data!  
Fiddler will **monitor this data and compare it to your baseline to generate powerful insights into how your model is behaving**.


---


Each record sent to Fiddler is called **an event**.
  
Let's load in some sample events from a CSV file.

In [15]:
PATH_TO_EVENTS_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/churn_events.csv'

production_df = pd.read_csv(PATH_TO_EVENTS_CSV)

# Shift the timestamps of the production events to be as recent as today 
production_df['timestamp'] = production_df['timestamp'] + (int(time.time() * 1000) - production_df['timestamp'].max())

In [16]:
print(production_df['timestamp'])

0      1687547982625
1      1687550411540
2      1687552840456
3      1687555269371
4      1687557698287
           ...      
245    1688143066962
246    1688145495878
247    1688147924793
248    1688150353709
249    1688152782625
Name: timestamp, Length: 250, dtype: int64


You can use the client's `publish_events_batch` function to start pumping data into Fiddler!
  
*Just include:*
1. The DataFrame containing your events
2. The name of the column containing event timestamps

In [17]:
client.publish_events_batch(
    project_id=PROJECT_ID,
    model_id=MODEL_ID,
    batch_source=production_df,
    timestamp_field='timestamp',
    id_field='customer_id' # Optional
)

{'status': 202,
 'job_uuid': 'd21bd1cd-bb61-426a-9aae-f882f527e1ed',
 'files': ['tmpsrjbqqvj.csv'],
 'message': 'Successfully received the event data. Please allow time for the event ingestion to complete in the Fiddler platform.'}

## 7. Get insights
  
Now just head to your Fiddler URL and start getting enhanced observability into your model's performance.

Run the following code block to get your URL.

In [18]:
print('/'.join([URL, 'projects', PROJECT_ID, 'models', MODEL_ID, 'monitor']))

https://preprod.fiddler.ai/projects/one_point_eight/models/churn_classifier/monitor


*Please allow 5-10 minutes for monitoring data to populate the charts.*
  
The following screen will be available to you upon completion.
<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/8.png" />
        </td>
    </tr>
</table>

**What's Next?**

Try the [NLP Monitoring - Quickstart Notebook](https://docs.fiddler.ai/docs/simple-nlp-monitoring-quick-start)

---


**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.

Join our [community Slack](http://fiddler-community.slack.com/) to ask any questions!

If you're still looking for answers, fill out a ticket on [our support page](https://fiddlerlabs.zendesk.com/) and we'll get back to you shortly.